# prepare

In [1]:
import pandas as pd
import numpy as np
import os
import seaborn as sns
import matplotlib.pyplot as plt
import scipy

from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import lightgbm as lgbm

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.float_format', lambda x: '%.4f' % x)

# Data Clean

In [2]:
def cutomizedCoordinationFix(df):
    df = df.assign(rev=df.dropoff_latitude<df.dropoff_longitude)
    idx = (df['rev'] == 1)
    df.loc[idx,['dropoff_longitude','dropoff_latitude']] = df.loc[idx,['dropoff_latitude','dropoff_longitude']].values
    df.loc[idx,['pickup_longitude','pickup_latitude']] = df.loc[idx,['pickup_latitude','pickup_longitude']].values
    df = df.drop(columns=['rev'])
    return df

def clean_df(df):    
    #reverse incorrectly assigned longitude/latitude values
    df = cutomizedCoordinationFix(df)
    df = df[(df.fare_amount > 0)  & (df.fare_amount <= 500) &
          (df.passenger_count >= 0) & (df.passenger_count <= 8)  &
           ((df.pickup_longitude != 0) & (df.pickup_latitude != 0) & (df.dropoff_longitude != 0) & (df.dropoff_latitude != 0) )]
    
    return df

# Use Featuretools to create feature

In [3]:
import featuretools as ft
print(f"featuretools version is {ft.__version__}")

from featuretools.primitives import TransformPrimitive
from woodwork.column_schema import ColumnSchema
from woodwork.logical_types import Double, LatLong, Datetime, Boolean

2022-12-02 23:27:48,028 featuretools - WARNING    Featuretools failed to load "nlp_primitives" primitives from "nlp_primitives". For a full stack trace, set logging to debug.
2022-12-02 23:27:48,037 featuretools - WARNING    Featuretools failed to load plugin nlp_primitives from library nlp_primitives. For a full stack trace, set logging to debug.
featuretools version is 1.18.0


# featuretools related function

In [4]:
from woodwork.logical_types import Ordinal

def produce_featuretools_entityset(es, df):
    trip_logical_types = {
        'passenger_count': Ordinal(order=list(range(0, 10))), 
        'pickup_latlong': 'LatLong',
        'dropoff_latlong': 'LatLong',
    }

    es.add_dataframe(dataframe_name="trips",
                     dataframe=df,
                     index="id",
                     time_index='pickup_datetime',
                     logical_types=trip_logical_types)

    return es


In [5]:
from featuretools.primitives import IsInGeoBox

class Bearing(TransformPrimitive):
    name = "bearing"
    input_types = [ColumnSchema(logical_type=LatLong), ColumnSchema(logical_type=LatLong)]
    return_type = ColumnSchema(logical_type=Double, semantic_tags={'numeric'})
    number_output_features = 1
    commutative=True
    def get_function(self):
        def bearing(latlong1, latlong2):
            lat1 = np.array([x[0] for x in latlong1])
            lon1 = np.array([x[1] for x in latlong1])
            lat2 = np.array([x[0] for x in latlong2])
            lon2 = np.array([x[1] for x in latlong2])
            delta_lon = np.radians(lon2 - lon1)
            lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])
            x = np.cos(lat2) * np.sin(delta_lon)
            y = np.cos(lat1) * np.sin(lat2) - np.sin(lat1) * np.cos(lat2) * np.cos(delta_lon)
            return np.degrees(np.arctan2(x, y))
        return bearing
    
class DistanceToLocation(TransformPrimitive):
    name = "distance_to_location"
    input_types = [ColumnSchema(logical_type=LatLong)]
    return_type = ColumnSchema(logical_type=Double, semantic_tags={'numeric'})
    number_output_features = 1
    commutative=True
    def __init__(self, point=(0, 0)):
        self.point = point
        self.lat = point[0]
        self.lon = point[1]
        
    def get_function(self):
        def distance_to_location(latlong):
            lat = np.array([x[0] for x in latlong])
            lon = np.array([x[1] for x in latlong])
            tgt_lat = len(lat) * self.lat
            tgt_lon = len(lon) * self.lon
            return self.sphere_dist(tgt_lat, tgt_lon, lat, lon)
        return distance_to_location
    
    def sphere_dist(self, lat1, lon1, lat2, lon2):
        """
        Return distance along great radius between pickup and dropoff coordinates.
        """
        #Define earth radius (km)
        R_earth = 6371
        #Convert degrees to radians
        lat1, lon1, lat2, lon2 = map(np.radians,[lat1, lon1, lat2, lon2])
        #Compute distances along lat, lon dimensions
        dlat = lat2 - lat1
        dlon = lon2 - lon1

        #Compute haversine distance
        a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2
        return 2 * R_earth * np.arcsin(np.sqrt(a))


def get_coordination(df):
    df["pickup_latlong"] = df[['pickup_latitude', 'pickup_longitude']].apply(tuple, axis=1)
    df["dropoff_latlong"] = df[['dropoff_latitude', 'dropoff_longitude']].apply(tuple, axis=1)
    df = df.drop(["pickup_latitude", "pickup_longitude", "dropoff_latitude", "dropoff_longitude"], axis = 1)
    return df

def modelling_features(df, feature_list = None, features_only = False):
    df = get_coordination(df)
    print(df.dtypes)

    es = ft.EntitySet("nyc_taxi_fare")
    es = produce_featuretools_entityset(es, df)
    
    cutoff_time = es['trips'][['id', 'pickup_datetime']]
    
    if feature_list:
        df = ft.calculate_feature_matrix(feature_list, entityset=es, cutoff_time=cutoff_time, verbose=True)
        return df, es, feature_list
    
    # airport coordination
    coordination_dicts = {
        "jfk_coord": (40.639722, -73.778889),
        "ewr_coord": (40.6925, -74.168611),
        "lga_coord": (40.77725, -73.872611),
        "sol_coord": (40.6892,-74.0445), # Statue of Liberty
        "nyc_coord": (40.7141667,-74.0063889) 
    }
    
    trans_primitives = ["day", "year", "month", "weekday", "haversine", "hour", "is_weekend", "is_working_hours", "part_of_day"]
    trans_primitives += ["cityblock_distance", Bearing,
                         IsInGeoBox((40.62, -73.85), (40.70, -73.75)),
                         IsInGeoBox((40.70, -73.97), (40.77, -73.9))]
    trans_primitives += [DistanceToLocation(x) for n, x in coordination_dicts.items()]

    # calculate feature_matrix using deep feature synthesis
    
    ret = ft.dfs(entityset=es,
                      target_dataframe_name="trips",
                      trans_primitives=trans_primitives,
                      verbose=True,
                      cutoff_time=cutoff_time,
                      approximate='36d',
                      max_depth=3,
                      max_features=40, 
                      features_only = features_only)
    if features_only:
        features = ret
    else:
        features = ret[1]
        df = ret[0]
        #df_encoded, features_encoded = ft.encode_features(df, features)
    
    return df, es, features

# Spark Pipeline

In [6]:
from pyspark import *
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import pandas_udf
from pyspark.sql.pandas.types import from_arrow_schema
import os
import re
import pyarrow as pa

def fix_df_types(df):
    categorical_cols = list(n for n, x in zip(df.dtypes.index.to_list(), df.dtypes.to_list()) if x.name == 'category')
    df[categorical_cols] = df[categorical_cols].apply(lambda col:pd.Categorical(col).codes)
    
    boolean_cols = list(n for n, x in zip(df.dtypes.index.to_list(), df.dtypes.to_list()) if x.name == 'boolean')
    df[boolean_cols] = df[boolean_cols].astype('int8')
    return df
    
def spark_modelling_features(df, feature_list = None, features_only = False):
    hname = os.uname()[1]
    # connect to spark
    spark = SparkSession.builder.master(f'spark://{hname}:7077')\
            .appName("featuretools_pandas_udf").getOrCreate()
    
    # create spark dataframe
    with Timer("create dataframe from pandas"):
        sparkDF = spark.createDataFrame(df)
    
    if feature_list:
        top_features = ft.load_features(feature_list)        
    else:
        top_features = None
        
    tmp_df, _, _ = modelling_features(df.head(100), feature_list = top_features, features_only = features_only)
    tmp_df = fix_df_types(tmp_df)
    return_schema = from_arrow_schema(pa.Schema.from_pandas(tmp_df, preserve_index=False))
    #print(return_schema)
        
    def pandas_udf_ft(iterator):
        for df in iterator:
            df, _, _ = modelling_features(df, feature_list = top_features, features_only = features_only)
            df = fix_df_types(df)
            print(df.dtypes)
            yield df 

    # execution
    sparkDF = sparkDF.mapInPandas(pandas_udf_ft, return_schema)
    try:
        df = sparkDF.toPandas()
    except:
        spark.stop()
    spark.stop()
    
    return df

In [7]:
# small data to test
from utils import Timer
from IPython.display import display

path = "../data/"
TRAIN_PATH = f'{path}/train.csv'
TEST_PATH = f'{path}/test.csv'

cols = [
    'fare_amount', 'pickup_datetime','pickup_longitude', 'pickup_latitude',
    'dropoff_longitude', 'dropoff_latitude', 'passenger_count'
]
 
#sampled_line = 100000
with Timer(f"Load train full"):
    train = pd.read_csv(TRAIN_PATH, usecols=cols)

with Timer("Data Wrangling for train"):
    train = clean_df(train)

train = spark_modelling_features(train)

display(train)

Load train full took 52.175250436645 sec
Data Wrangling for train took 7.914470876101404 sec


Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/12/02 23:29:01 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/12/02 23:29:01 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).


create dataframe from pandas took 1478.8554169246927 sec
fare_amount        float64
pickup_datetime     object
passenger_count      int64
pickup_latlong      object
dropoff_latlong     object
dtype: object
Built 27 features
Elapsed: 00:01 | Progress: 100%|██████████


22/12/02 23:53:45 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
22/12/02 23:53:46 WARN TaskSetManager: Stage 0 contains a task of very large size (55949 KiB). The maximum recommended task size is 1000 KiB.


,fare_amount,passenger_count,"BEARING(dropoff_latlong, pickup_latlong)","CITYBLOCK_DISTANCE(dropoff_latlong, pickup_latlong)",DAY(pickup_datetime),"DISTANCE_TO_LOCATION(dropoff_latlong, point=(40.639722, -73.778889))","DISTANCE_TO_LOCATION(pickup_latlong, point=(40.639722, -73.778889))","DISTANCE_TO_LOCATION(dropoff_latlong, point=(40.6892, -74.0445))","DISTANCE_TO_LOCATION(pickup_latlong, point=(40.6892, -74.0445))","DISTANCE_TO_LOCATION(dropoff_latlong, point=(40.6925, -74.168611))",...,"IS_IN_GEOBOX(dropoff_latlong, point1=(40.62, -73.85), point2=(40.7, -73.75))","IS_IN_GEOBOX(pickup_latlong, point1=(40.62, -73.85), point2=(40.7, -73.75))","IS_IN_GEOBOX(dropoff_latlong, point1=(40.7, -73.97), point2=(40.77, -73.9))","IS_IN_GEOBOX(pickup_latlong, point1=(40.7, -73.97), point2=(40.77, -73.9))",IS_WEEKEND(pickup_datetime),IS_WORKING_HOURS(pickup_datetime),MONTH(pickup_datetime),PART_OF_DAY(pickup_datetime),WEEKDAY(pickup_datetime),YEAR(pickup_datetime)
0,10.2000,2,51.5815,3.2705,0,16845.7890,16844.8699,16064.6075,16063.4038,15685.9796,...,0,0,0,0,0,0,0,5,3,2008
1,8.5000,2,162.8423,2.1718,0,16847.3142,16845.0378,16065.8207,16063.6797,15687.2173,...,0,0,0,1,0,1,0,0,3,2008
2,6.5000,2,81.8782,1.3039,0,16847.1342,16845.8333,16065.6988,16064.2975,15687.0909,...,0,0,0,0,0,1,0,3,3,2008
3,5.8000,1,36.7586,1.6617,1,16846.8419,16846.8622,16065.5191,16065.3881,15686.9023,...,0,0,0,0,0,0,0,5,4,2008
4,5.3000,1,7.0654,1.3152,1,16845.9892,16846.9470,16064.7200,16065.5452,15686.0989,...,0,0,0,0,0,0,0,2,4,2008
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54315950,7.5000,2,-151.3658,1.7654,26,8520.3917,8520.6260,9361.6764,9362.0746,9679.6977,...,0,0,0,0,1,0,5,1,5,2014
54315951,14.5000,5,39.1246,3.9868,27,8521.1360,8521.4605,9362.7960,9362.7597,9681.0525,...,0,0,0,0,1,0,5,5,6,2014
54315952,3.5000,1,91.4539,0.0639,29,8521.1622,8521.2458,9362.3525,9362.4313,9680.3114,...,0,0,0,0,0,1,5,0,1,2014
54315953,12.0000,1,-141.9737,1.8517,29,8520.5307,8520.4209,9361.6544,9361.7128,9679.5732,...,0,0,0,0,0,1,5,0,1,2014


# EvalML Train

In [ ]:
train = pd.read_parquet("featuretools_process_nyc_taxi.parquet")
train = train.head(10000000)

def get_split_sets(train):
    x = train.drop(columns=['fare_amount'])
    y = train['fare_amount'].values
    x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.1, random_state=123)
    return x_train, x_val, y_train, y_val

with Timer("split train and val"):
    x_train, x_val, y_train, y_val = get_split_sets(train)
    
# looking for right ml pipeline
import evalml
from evalml import AutoMLSearch

automl = AutoMLSearch(X_train=x_train,
                      y_train=y_train,
                      X_holdout=X_val,
                      y_holdout=y_val,
                      problem_type="regression",
                      objective="root mean squared error",
                      verbose=True,)
automl.search()

best_pipeline = automl.best_pipeline
with Timer("train"):
    best_pipeline.fit(x_train, y_train)
    
best_pipeline.score(X_val, y_val, objectives=["root mean squared error"])

# LGBM Train

In [43]:
import re

train = pd.read_parquet("featuretools_process_nyc_taxi.parquet")
train = train.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
print(train.dtypes)

def get_split_sets(train):
    x = train.drop(columns=['fare_amount'])
    y = train['fare_amount'].values
    x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.1, random_state=0)
    return x_train, x_val, y_train, y_val

with Timer("split train and val"):
    x_train, x_val, y_train, y_val = get_split_sets(train)

params = {
        'boosting_type':'gbdt',
        'objective': 'regression',
        'nthread': 4,
        'num_leaves': 31,
        'learning_rate': 0.05,
        'max_depth': -1,
        'subsample': 0.8,
        'bagging_fraction' : 1,
        'max_bin' : 5000 ,
        'bagging_freq': 20,
        'colsample_bytree': 0.6,
        'metric': 'rmse',
        'min_split_gain': 0.5,
        'min_child_weight': 1,
        'min_child_samples': 10,
        'scale_pos_weight':1,
        'zero_as_missing': True,
        'seed':0,
        'num_rounds':1000,
        'num_boost_round': 10000,
        'early_stopping_rounds': 50
    }


# lgbm_train = lgbm.Dataset(x_train, y_train, silent=False, categorical_feature=['passenger_count','year','time','dayofyear','weekday'])
# lgbm_val = lgbm.Dataset(x_val, y_val, silent=False, categorical_feature=['passenger_count','year','time','dayofyear','weekday'])

lgbm_train = lgbm.Dataset(x_train, y_train, silent=False)
lgbm_val = lgbm.Dataset(x_val, y_val, silent=False)


with Timer("train"):
    model = lgbm.train(params=params, train_set=lgbm_train, valid_sets=lgbm_val, verbose_eval=100)
    
with Timer("predict"):
    pred = model.predict(x_val, num_iteration=model.best_iteration)
    
with Timer("calculate rmse"):
    rmse = np.sqrt(mean_squared_error(y_val, pred))

print('LightGBM RMSE', rmse)

fare_amount                                   float64
passenger_count                                 int64
GEOBOXdropoff_latlong406273854077375             bool
GEOBOXpickup_latlong406273854077375              bool
GEOBOXdropoff_latlong40773974077739              bool
GEOBOXpickup_latlong40773974077739               bool
BEARINGdropoff_latlongpickup_latlong          float64
CITYBLOCKdropoff_latlongpickup_latlong        float64
IS_NIGHT_HOURpickup_datetime                     bool
IS_NOON_HOURpickup_datetime                      bool
IS_RUSH_HOURpickup_datetime                      bool
passenger_cntIS_NIGHT_HOURfirst_trips_time       bool
passenger_cntIS_NOON_HOURfirst_trips_time        bool
passenger_cntIS_RUSH_HOURfirst_trips_time        bool
dtype: object
split train and val took 15.84280596114695 sec
[LightGBM] [Warning] bagging_fraction is set=1, subsample=0.8 will be ignored. Current value: bagging_fraction=1
[LightGBM] [Warning] bagging_fraction is set=1, subsample=0.8 will be 

# LGBM for 10 million sample

In [12]:
with Timer("split train and val"):
    x_train, x_val, y_train, y_val = get_split_sets(train)

lgbm_params = {
    'objective': 'regression',
    'boosting': 'gbdt',
    'reg_sqrt': True,
    'learning_rate': 0.03,
    'num_leaves': 1200,
    'max_depth': -1,
    'max_bin': 5000,
    'num_rounds': 1200,
    'early_stopping_round': 50,
    'metric': 'rmse'
}

lgbm_train = lgbm.Dataset(x_train, y_train, silent=False)
lgbm_val = lgbm.Dataset(x_val, y_val, silent=False)

with Timer("train"):
    model = lgbm.train(params=lgbm_params, train_set=lgbm_train, valid_sets=lgbm_val, verbose_eval=100)
    
with Timer("predict"):
    pred = model.predict(x_val, num_iteration=model.best_iteration)
    
with Timer("calculate rmse"):
    rmse = np.sqrt(mean_squared_error(y_val, pred))

print('LightGBM RMSE', rmse)

split train and val took 3.7570005068555474 sec
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.129873 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 30536
[LightGBM] [Info] Number of data points in the train set: 9000000, number of used features: 12
[LightGBM] [Info] Start training from score 3.177760
Training until validation scores don't improve for 50 rounds
[100]	valid_0's rmse: 4.25898
[200]	valid_0's rmse: 4.12228
[300]	valid_0's rmse: 4.08928
[400]	valid_0's rmse: 4.0765
[500]	valid_0's rmse: 4.07053
[600]	valid_0's rmse: 4.06616
[700]	valid_0's rmse: 4.06431
[800]	valid_0's rmse: 4.06246
[900]	valid_0's rmse: 4.06072
[1000]	valid_0's rmse: 4.05996
[1100]	valid_0's rmse: 4.05938
Early stopping, best iteration is:
[1107]	valid_0's rmse: 4.05933
train took 803.9622147129849 sec
predict took 4.348827651701868 sec
calculate rmse

# EDA

In [ ]:
def exploration_features(df):
    """adds features for use in the EDA section"""
    df = shared_features(df)
    df = (
        df
        .assign(
            hour=df.pickup_datetime.dt.hour,
            close_to_airport='No',
            fare_per_km=df.fare_amount*1000/df.distance,
            direction_bucket = pd.cut(df.direction, np.linspace(-180, 180, 37)),

            #small location buckets
            pickup_long_bucket=pd.cut(df.pickup_longitude, bins=2550, labels=False),
            pickup_lat_bucket=pd.cut(df.pickup_latitude, bins=2200, labels=False),
            dropoff_long_bucket=pd.cut(df.dropoff_longitude, bins=2550, labels=False),
            dropoff_lat_bucket=pd.cut(df.dropoff_latitude, bins=2200, labels=False),


            #large location buckets
            pickup_long_bucket_big=pd.cut(df.pickup_longitude, bins=255, labels=False),
            pickup_lat_bucket_big=pd.cut(df.pickup_latitude, bins=220, labels=False),
            dropoff_long_bucket_big=pd.cut(df.dropoff_longitude, bins=255, labels=False),
            dropoff_lat_bucket_big=pd.cut(df.dropoff_latitude, bins=220, labels=False)
        )
        .drop(columns='pickup_datetime')
        .query("0 < distance")
    )

    df.loc[((df['pickup_dist_jfk']<1500) | (df['dropoff_dist_jfk']<1500)), 'close_to_airport'] = 'JFK'
    df.loc[((df['pickup_dist_lga']<1500) | (df['dropoff_dist_lga']<1500)), 'close_to_airport'] = 'LaGuardia'
    df.loc[((df['pickup_dist_nla']<1500) | (df['dropoff_dist_nla']<1500)), 'close_to_airport'] = 'Newark'  
    return df

